In [ ]:
w1^2 sigma1^2 + (1-w1)^2 sigma2^2 + 2 sigma1 sigma2 w1 (1-w1) <= 1/9.55 (3.09(w1^2 sigma1^2 + (1-w1)^2 sigma2^2 + 2 sigma1 sigma2 w1 (1-w1) ) (mu1 w1 + mu2(1-w1) Wt))^2

In [3]:
from scipy.optimize import minimize

# Define the objective function to minimize
def objective_function(x):
    return (x[0] - 2) ** 2 + (x[1] - 3) ** 2

# Define the constraints (inequality constraint: x[0] + x[1] >= 1)
def constraint_function(x):
    return x[0] + x[1] - 1

# Initial guess for the optimization
initial_guess = [0, 0]

# Define the bounds for the variables
bounds = ((-10, 10), (-10, 10))

# Define the constraint dictionary
constraints = ({'type': 'ineq', 'fun': constraint_function})

# Perform the optimization
result = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints)

# Print the result
print("Optimal solution:")
print("x:", result.x)
print("Optimal value:", result.fun)


Optimal solution:
x: [2. 3.]
Optimal value: 0.0


In [5]:
from scipy.stats import norm
norm.ppf(0.999)

3.090232306167813

In [8]:
from scipy.optimize import minimize
from scipy.stats import norm
import numpy as np

def objective_function(w, returns, initial_wealth):
    return - (w[0] * returns[0] + w[1] * returns[1]) * initial_wealth


def weight_sum_constraint(w):
    return w[0] + w[1] - 1.0


def expected_value_portfolio(w, returns, initial_wealth):
    return (w[0] * returns[0] + w[1] * returns[1]) * initial_wealth


def variance_portfolio(w, vcv_matrix):
    return w @ vcv_matrix @ w


def calculate_var(w, quantile, vcv_matrix, returns, initial_wealth):
    variance = variance_portfolio(w, vcv_matrix)
    expected_value_portfolio_number = expected_value_portfolio(w, returns, initial_wealth)
    return np.sqrt(variance) * norm.ppf(quantile) + expected_value_portfolio_number


def var_risk_constraint(w, quantile, vcv_matrix, returns, initial_wealth):
    variance = variance_portfolio(w, vcv_matrix)
    var = calculate_var(w, quantile, vcv_matrix, returns, initial_wealth)
    return variance - var / norm.ppf(quantile)


def calculate_optimal_weights(returns, vcv_matrix, initial_wealth, var_quantile):
    """This function calculates the optimal weights for an investor maximizing return
    under a Value-at-Risk constraint. The market has only 2 risky assets

    Args:
        returns (np.array): A numpy array of expected returns for each asset
        vcv_matrix (np.array): A numpy array of the variance-covariance matrix for the returns od the assets!
        initial_wealth (float): The initial wealth of the investor
        var_quantile (float): The quantile of Value-at-Risk constraint

    Returns:
        optimal_weights (np.array): A numpy array of the optimal weights for each asset
    """

    # Initial guess for the optimization
    initial_guess = [0.5, 0.5]

    # Constraint Dictionary
    constraints = (
        {'type': 'eq', 'fun': weight_sum_constraint},
        {'type': 'ineq', 'fun': var_risk_constraint, 'args': (var_quantile, vcv_matrix, returns, initial_wealth)})

    # Perform optimization
    result = minimize(objective_function, initial_guess, constraints=constraints, args=(returns, initial_wealth))

    return result.x


# Check if the script is being run as the main program
if __name__ == "__main__":
    returns = np.array([1.1, 1.2])
    vcv_matrix = np.array([[1.1, 0.5], [0.5, 1.5]])
    initial_wealth = 100
    var_quantile = 0.99

    optimal_weights = calculate_optimal_weights(returns, vcv_matrix, initial_wealth, var_quantile)

    print(optimal_weights)


[ 5.24102686 -4.24102686]
